# The reproduction number
The reproduction number is a key quantity in infectious disease modelling,
which has major implications for epidemic dynamics.
It represents the average number of secondary infectious persons resulting from a single infectious person.
In modelling, we often consider both the time-varying reproduction (_R<sub>t</sub>_)
and the basic reproduction number (_R<sub>0</sub>_).
The former (_R<sub>t</sub>_) represents the number of secondary infectious being produced at a time point
in the epidemic or the simulation we are running,
whereas the latter (_R<sub>0</sub>_) represents the theoretical number of secondary
infectious cases that would occur from a single infectious person in a fully susceptible population.

The basic reproduction number is a useful way of conceptualising the overall infectiousness of a pathogen,
and provides additional information to the "contact rate" that we have previously introduced.
The key distinction is that the contact rate represents the number of secondary infections
resulting **per unit time**, whereas the reproduction number is the number of secondary
occurring infectious cases **per infectious case**.

In [ ]:
# If running on Google Colab, run the following line of code to install the summer package
# %pip install summerepi2

In [ ]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def build_sir_model(
    model_config: dict,
) -> CompartmentalModel:
    """
    This model is almost identical to the one introduced in notebook 02,
    except even simpler because the death outflow has been removed.
    """
    compartments = (
        "susceptible",
        "infectious",
        "recovered",
    )
    analysis_times = (
        model_config["start_time"], 
        model_config["end_time"],
    )
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=["infectious"],
    )
    model.set_initial_population(
        distribution=
        {
            "susceptible": model_config["population"] - model_config["seed"], 
            "infectious": model_config["seed"],
        }
    )
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("contact_rate"),
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("recovery"),
        source="infectious", 
        dest="recovered",
    )
    return model

In [ ]:
config = {
    "population": 1000.,
    "seed": 10.,
    "start_time": 0.,
    "end_time": 20.,
}

sir_model = build_sir_model(config)

We only need one parameter value now,
because we don't have deaths any more and we're going to set the contact rate
by calculating this later from the _R<sub>0</sub>_ value we're targeting.

In [ ]:
parameters = {
    "recovery": 0.333,
}
sojourn_infectious = 1. / parameters["recovery"]

In [ ]:
def get_output_from_r0s(
    model: CompartmentalModel,
    basic_reproduction_numbers: tuple,
    compartment: str,
) -> pd.DataFrame:
    """
    Run the model with various basic reproduction numbers,
    provided as a user input, and collate the results together.
    
    Args:
        model: The model to be run
        basic_reproduction_numbers: The reproduction numbers to use
        compartment: The compartment output of interest
    Returns:
        outputs: Dataframe with the compartment outputs 
            in columns for each reproduction number
    """
    outputs = pd.DataFrame(columns=basic_reproduction_numbers)
    for r0 in basic_reproduction_numbers:
        parameters["contact_rate"] = r0 / sojourn_infectious
        model.run(parameters=parameters)
        outputs[r0] = sir_model.get_outputs_df()[compartment]
    return outputs

## Effect of _R<sub>0</sub>_
Of course, the basic reproduction number has a major effect on epidemic dynamics.
As for the contact rate, the infectiousness of the pathogen we are simulating
scales directly with the parameter value we use for this quantity.
A higher _R<sub>0</sub>_ drives a more rapid take off of the epidemic during the early exponential growth phase.
However, this also leads to a more rapid depletion of the susceptible pool,
so that the epidemic also begins to decline sooner.

In [ ]:
high_r0s = (1., 2., 5., 10.)
get_output_from_r0s(sir_model, high_r0s, "infectious").plot(
    labels={"index": "time", "value": "number infectious"},
)

## Threshold value for epidemic take-off
The threshold value of one for the reproduction number is a key tipping point, 
above which we can expect the epidemic to take off,
but below which we expect the infection to steadily decline in the community.
Because we are not concerned with stochastic (random) extinction events in these deterministic models,
this is seen consistently around this threshold value of one.
(One small qualifier here is that for this numerical solution,
the threshold doesn't occur exactly at one,
because even the fact that a small proportion of the population start off
infectious marginally reduces the proportion of the population that is susceptible to infection.)

In [ ]:
config.update(
    {"end_time": 100.}
)
sir_model = build_sir_model(config)
close_1_r0s = (0.9, 1.1)
get_output_from_r0s(sir_model, close_1_r0s, "infectious").plot(
    labels={"index": "time", "value": "number infectious"},
)

## Epidemic final size
The higher we turn _R<sub>0</sub>_,
the greater the proportion of the population that will be infected.
As we reach very high values for this quantity (e.g. >10),
nearly all the population becomes infected by the end of the simulation period
(although always short of 

In [ ]:
final_size_r0s = np.linspace(1., 4., 31)
config.update(
    {
        "end_time": 200.,
        "population": 1.,
        "seed": 0.01,
    }
)
sir_model = build_sir_model(config)
get_output_from_r0s(sir_model, final_size_r0s, "recovered").iloc[-1].plot(
    labels={"index": "R0", "value": "epidemic final size"},
).update_layout(showlegend=False)

## The time-varying reproduction number, _R<sub>t</sub>_
With our simple epidemic model, if the entire population is susceptible,
the effective reproduction number will be equal to the the basic reproduction number.
However, as the proportion of the population susceptible declines,
the effective reproduction number will also fall.
Specifically, the time-varying reproduction number is scaled by 
the proportion of the population remaining susceptible, such that
$$ R_{t}= R_{0}\times \frac{S(t)}{N(t)} $$
where $ N(t) $ is the population size and $ S(t) $ is the number of susceptibles at time $ t $.
As the proportion of the population susceptible declines below $ \frac{1}{R_{0}} $,
we expect the effective reproduction number to fall below one.
This will result in the epidemic no longer increasing and instead beginning to decline.
Let's see if that happens.

In [ ]:
# Just scale a couple of configuration parameters to allow for easier inspection
config.update(
    {
        "end_time": 20.,
        "population": 10.,   
    }
)
sir_model = build_sir_model(config)

In [ ]:
# Chose your basic reproduction number
r0 = 4.

# Calculate the contact rate parameter from R0
parameters.update(
    {
        "contact_rate": r0 / sojourn_infectious,
    }
)

# Run and get outputs
sir_model.run(parameters=parameters)
outputs = sir_model.get_outputs_df()

# Calculate the time-varying reproduction number, Rt
outputs["reproduction_number"] = r0 * outputs["susceptible"] / config["population"]

# Note the threshold value of one
outputs["threshold"] = 1.

# Plot
outputs[["reproduction_number", "infectious", "threshold"]].plot(
    labels={"index": "time", "value": "number or prevalence"}
)

Note that although this is true for our very simple SIR model,
including a latent period to create an SEIR model 
would result in a small delay between the 
reproduction number falling to one and the reversal of the epidemic.

## Herd immunity threshold
The herd immunity threshold is the point at which 
a sufficient proportion of the population have achieved immunity
for the epidemic to begin to decline.
When the proportion susceptible is the reciprocal of 
the basic reproduction number, population immunity will be sufficient to slow transmission.
In mathematical terms, at the herd immunity threshold:
$$ R_{t} = 1 $$
$$ R_{t} = R_{0} \times S(t) $$
$$ S(t) \approx 1 - R(t) $$
From these equations, we have:
$$ R_{0} \times S(t) = 1 $$
$$ S(t) = \frac{1}{R_{0}} $$
$$ R(t) = 1 - \frac{1}{R_{0}} $$
This is a class calculation of infectious disease modelling.